In [46]:
from torch import nn
import torch
from torch.nn import functional as F
from scipy.fftpack import fft,ifft
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [8]:
net=MLP()
X=torch.normal(size=(10,20),mean=0,std=1)
net(X)

tensor([[-0.2134, -0.2015, -0.1919, -0.3539, -0.0152, -0.1017, -0.0842,  0.1741,
         -0.0606,  0.3860],
        [-0.4065,  0.0288, -0.2297, -0.6464,  0.1713,  0.2619, -0.3145, -0.1225,
         -0.0079,  0.3345],
        [-0.1719, -0.1638, -0.0548, -0.3054,  0.0793, -0.2450, -0.0977, -0.0243,
          0.1042, -0.0279],
        [-0.0655,  0.1287, -0.4343,  0.3586,  0.0254,  0.2035, -0.1163,  0.1022,
         -0.2928,  0.0995],
        [-0.2721, -0.1784,  0.0287, -0.3653,  0.0362, -0.1364, -0.0938,  0.1319,
         -0.0185,  0.2145],
        [-0.0247,  0.0035,  0.0427, -0.0395,  0.0997, -0.0703, -0.0635, -0.2523,
         -0.3468,  0.1082],
        [-0.1827,  0.3742,  0.0087, -0.5211,  0.1574,  0.4310,  0.2323, -0.0228,
         -0.2131,  0.3930],
        [ 0.1469, -0.0903, -0.3157, -0.0538, -0.0977,  0.0048, -0.2533,  0.3424,
          0.0045, -0.0344],
        [-0.1134, -0.2968,  0.1536, -0.1223,  0.3515,  0.0456, -0.2045,  0.0529,
         -0.0310,  0.0527],
        [-0.1269, -

In [11]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block]=block
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

In [12]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1152,  0.0811, -0.1166,  0.3599, -0.0778, -0.1648, -0.1564, -0.1097,
          0.0263,  0.3762],
        [-0.1831,  0.4942, -0.3682, -0.1045, -0.3149, -0.1770, -0.1376,  0.2979,
         -0.1573,  0.5644],
        [ 0.1419,  0.3288, -0.2088,  0.0513, -0.0448, -0.0019, -0.0485, -0.0769,
          0.1604,  0.3819],
        [-0.3013, -0.0275,  0.0242, -0.2029,  0.3036,  0.1372,  0.3079,  0.2586,
          0.2608,  0.3015],
        [-0.0786,  0.4147, -0.1808,  0.2438,  0.3697, -0.4498, -0.1408, -0.0570,
          0.1732,  0.4524],
        [ 0.1615,  0.5546, -0.1168, -0.1011,  0.0939,  0.1413,  0.0831,  0.0292,
          0.2830,  0.2039],
        [ 0.3220, -0.3814,  0.0879,  0.3373,  0.0428, -0.1796, -0.4034,  0.2381,
          0.6042,  0.4652],
        [-0.4978,  0.2606, -0.3775,  0.0086, -0.0580, -0.4430,  0.1749, -0.2599,
          0.2674, -0.1226],
        [ 0.5911,  0.0011, -0.0389,  0.3245, -0.2886, -0.1969,  0.0315,  0.2182,
          0.4998,  0.1438],
        [ 0.3467,  

In [3]:
class myblock(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block]=block
    def foward(self,X):
        for block in self._modules.value():
            X=block(X)
        return X

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))


In [14]:
print(rgnet[0][0][0].bias)

Parameter containing:
tensor([ 0.0114, -0.1055,  0.2234, -0.0604,  0.4429,  0.3607, -0.4041,  0.2968],
       requires_grad=True)


In [71]:
import numpy as np
class disdimension(nn.Module):
    def __init__(self,input_n:int,output_n:int):
        super().__init__()
        self.W=torch.normal(size=(input_n,input_n,output_n),mean=0,std=1)
        
    def forward(self,X):
        return torch.matmul(torch.matmul(X.T,self.W).T,X)
class fftpar(nn.Module):
    def __init__(self,input_n:int):
        super().__init__()
    def forward(self,X):
        n=len(X)
        ans=abs(fft(np.array(X)))
        print(type(ans),ans.shape)
        return torch.Tensor(ans)[0:int(n/2),0]

In [72]:
POD=disdimension(10,5)
X=torch.normal(size=(10,1),mean=0,std=1)
fftl=fftpar(10)
print(X,fftl(X))

<class 'numpy.ndarray'> (10, 1)
tensor([[ 0.4173],
        [ 0.2864],
        [ 1.3169],
        [ 0.0218],
        [-0.4808],
        [-1.1164],
        [-0.0430],
        [ 0.3189],
        [ 1.1186],
        [ 0.3859]]) tensor([0.4173, 0.2864, 1.3169, 0.0218, 0.4808])


In [78]:
net=disdimension(10,5)
torch.save(net.state_dict(),'netpar.txt')
newnet=disdimension(10,5)
newnet.load_state_dict(torch.load('netpar.txt'))

<All keys matched successfully>

In [79]:
print(newnet)

disdimension()
